In [16]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
import requests

from tqdm import tqdm
import time

import itertools

In [4]:
link = "https://stock-pictures.netlify.app"

In [6]:
browser = webdriver.Edge()

In [10]:
browser.get(link)

In [12]:
soup = BeautifulSoup(browser.page_source, "html.parser")

In [14]:
soup.text

'Stock ImagesYou need to enable JavaScript to run this app.Stock Images with TagsTags - Clouds, Sky, Atmosphere, Blue Sky196 Likes55 Comments Tags - Bird, Ornithology, Hummingbird76 Likes20 Comments Tags - Sea, Rainbow, Rainfall, Subtropical282 Likes106 Comments Tags - Cherry Blossoms, Road, Japan, Sakura42 Likes11 Comments Tags - Cape Marguerite, Flower, Plant39 Likes15 Comments Tags - Relaxing Rest Under The Tree522 Likes108 Comments Tags - Cottontail Rabbit, Wild Rabbit, Grass51 Likes10 Comments Tags - Crocus, Flowers, Spring, Plant135 Likes44 Comments Tags - Coffee, Café, Vacation, Drink, Table26 Likes5 Comments Tags - Waterfall, Fall, Epic, Nature, Light96 Likes21 Comments Tags - Field, Morning, Sunrise, Dawn, Nature807 Likes131 Comments Tags - Bird, Robin, Birdwatching, Animal69 Likes19 Comments Tags - Seedlings, Soil, Spring Flowers, Bloom42 Likes9 Comments Tags - Road, Highway, Countryside, Hills53 Likes14 Comments Tags - Cityscape, Monochrome, Bridge, City45 Likes13 Comments T

## automate auto scroling to end

In [33]:
#itertools enable to iterate till infinity
for i in tqdm(itertools.count(1, 10000)):
    browser.execute_script("window.scrollTo(0," + str(i)+ ")")
    time.sleep(0.2)

1170it [04:31,  4.30it/s]


KeyboardInterrupt: 

## finding the bottom

In [35]:
for i in tqdm(range(0,5000000,1000)):
    
    browser.execute_script("window.scrollTo(0,"  + str(i) + ")")
    time.sleep(.1)

100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [09:38<00:00,  8.64it/s]


## scraping the page

In [37]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [40]:
len(soup.find_all('div', class_ = 'container'))

9104

In [42]:
data = []

for sp in tqdm(soup.find_all('div', class_ = 'container')):

    img_link = sp.find('img').get('src')
    tags     = sp.find('span', class_ = 'tag-color').text[7:].strip()
    likes    = int(sp.find('div', class_ = 'likes-comments').find_all('span')[0].text.strip()[:-6])
    comments = int(sp.find('div', class_ = 'likes-comments').find_all('span')[1].text.strip()[:-9])
    
    data.append([img_link, tags, likes, comments])

100%|████████████████████████████████████████████████████████████████████████████| 9104/9104 [00:00<00:00, 9653.59it/s]


## saving data in dataframe

In [45]:
df = pd.DataFrame(data, columns = ['img_link', 'tags', 'likes', 'comments'])

df.head()

,img_link,tags,likes,comments
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Clouds, Sky, Atmosphere, Blue Sky",196,55
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Ornithology, Hummingbird",76,20
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Sea, Rainbow, Rainfall, Subtropical",282,106
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Cherry Blossoms, Road, Japan, Sakura",42,11
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Cape Marguerite, Flower, Plant",39,15


## checking null values

In [48]:
df.isnull().sum()

img_link    0
tags        0
likes       0
comments    0
dtype: int64

## checking for duplicate links

In [51]:
df['img_link'].describe()

count                                                  9104
unique                                                 9088
top       https://cdn.pixabay.com/photo/2022/03/06/05/30...
freq                                                      2
Name: img_link, dtype: object

## removing duplicates

In [54]:
df = df.drop_duplicates()

In [56]:
df.to_csv('images.csv', index = False)

In [12]:
df= pd.read_csv("images.csv")

## dynamic naming

In [2]:
ref = 100000
for i in range(1,11):  
    print('A' + str(i + ref)[1:])

A00001
A00002
A00003
A00004
A00005
A00006
A00007
A00008
A00009
A00010


## downloading images

In [6]:
def download(link, img_path ):

    res = requests.get(link)         # Sending the Request
    fd = open(img_path,'wb')         # Getting Image in WB Format
    fd.write(res.content)            # Saving the Image
    fd.close()

In [18]:
paths   = []
img_ids = []

ref = 100000
i   = 1

for link in tqdm(df['img_link']):
    
    img_id = 'A' + str(i + ref)[1:]          # Creating Image ID
    i += 1
    
    img_path = 'Imgs/' + img_id + '.jpg'    # Creating Image Path
    
    download(link, img_path)                # Downloading the Image
    
    paths.append(img_path)
    img_ids.append(img_id)

100%|██████████████████████████████████████████████████████████████████████████████| 9101/9101 [38:14<00:00,  3.97it/s]


In [20]:
df['img_id']   = img_ids
df['img_path'] = paths

,img_link,tags,likes,comments,img_id,img_path
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Clouds, Sky, Atmosphere, Blue Sky",196,55,A00001,Imgs/A00001.jpg
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Ornithology, Hummingbird",76,20,A00002,Imgs/A00002.jpg
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Sea, Rainbow, Rainfall, Subtropical",282,106,A00003,Imgs/A00003.jpg
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Cherry Blossoms, Road, Japan, Sakura",42,11,A00004,Imgs/A00004.jpg
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Cape Marguerite, Flower, Plant",39,15,A00005,Imgs/A00005.jpg


In [22]:
df

,img_link,tags,likes,comments,img_id,img_path
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Clouds, Sky, Atmosphere, Blue Sky",196,55,A00001,Imgs/A00001.jpg
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Ornithology, Hummingbird",76,20,A00002,Imgs/A00002.jpg
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Sea, Rainbow, Rainfall, Subtropical",282,106,A00003,Imgs/A00003.jpg
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Cherry Blossoms, Road, Japan, Sakura",42,11,A00004,Imgs/A00004.jpg
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Cape Marguerite, Flower, Plant",39,15,A00005,Imgs/A00005.jpg
...,...,...,...,...,...,...
9096,https://cdn.pixabay.com/photo/2018/01/08/18/51...,"Pear, Fruit, Food, Background, Healthy",75,11,A09097,Imgs/A09097.jpg
9097,https://cdn.pixabay.com/photo/2021/09/22/01/42...,"Ants, Insects, Plants, Flowers, Animal",13,8,A09098,Imgs/A09098.jpg
9098,https://cdn.pixabay.com/photo/2022/01/09/15/58...,"Flower, Flora, Garden, Nature",0,0,A09099,Imgs/A09099.jpg
9099,https://cdn.pixabay.com/photo/2021/08/10/11/59...,"Kara Dag Mountain, Resort Village, Town",6,2,A09100,Imgs/A09100.jpg


In [26]:
df.to_csv('imgs.csv', index = False)